# Q1

In [1]:
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [2]:
import numpy as np
original = np.arange(0.0, 10.0, 0.01, dtype='f4')
shuffled = original.copy()
np.random.shuffle(shuffled)

In [3]:
sorted = shuffled.copy()
bubblesort(sorted)
print(np.array_equal(sorted, original))

True


In [4]:
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)


182 ms ± 4.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
#Complete the Solution to Q1 (i) here.
from numba import jit 
@jit
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [6]:
#Measure the time of your optimized implementation by running this cell

%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

638 µs ± 2.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
#Complete the Solution to Q1 (i) here.
#@jit(void(f4[:]))
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [ ]:
#Measure the time of your optimized implementation by running this cell

# Q2.

In [8]:
from scipy.stats import pareto
import matplotlib.pyplot as plt

import numpy as np
import math
from numba import vectorize,float64, cuda

fig, ax = plt.subplots(1, 1)

In [9]:
b=2.62
#x = np.linspace(1.0, 5.7, 100)

x = np.linspace(1.0, 5.7, 1000000) # Taking this value will give the proper time difference.

In [13]:
#Run this cell to know the time taken by original scipy implementation.

%timeit pareto.pdf(x, b)

116 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
# Write your code here for CPU_BASED optimization. Give new name as pareto_new_cpu()

@vectorize
def pareto_new_cpu(x,b):
    return b/(x**(b+1))

In [16]:
# Run this cell to check the performance of above implementation.

%timeit pareto_new_cpu(x,b)

60.4 ms ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
# Write your code here for GPU_BASED optimization. Give new name as pareto_new_gpu()
@vectorize([float64(float64, float64)], target='cuda') 
def pareto_new_gpu(x, b):
  return b/(x**(b+1))

In [18]:
%%timeit 
pareto_new_gpu(x,b)

9.11 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Q3

In [46]:
# Create on device memory variables here.
def pareto_new(x_dev, b_dev, out=out_dev):
    b = 2.62
    n = 100000
    x = np.linspace(1.0, 5.7, n)
    b_dev = cuda.to_device(b)
    x_dev = cuda.to_device(x)
    out_dev = cuda.device_array(shape = (n,), dtype=np.float64)

In [47]:
%timeit pareto_new(x_dev, b_dev, out=out_dev)

946 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [48]:
out_host = out_dev.copy_to_host() # copy the output variable from device to host

In [49]:
out_host.shape

(100000,)

In [50]:
ax.plot(x, out_host,'r-')